In [61]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [62]:
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import accuracy_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [63]:
df = pd.read_csv('train1.csv', usecols = ['Age', 'Fare', 'Survived'])

In [64]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [65]:
df.shape

(891, 3)

In [66]:
df.isnull().sum()

Survived      0
Age         177
Fare          0
dtype: int64

In [67]:
df.dropna(inplace=True)

In [68]:
df.isnull().sum()

Survived    0
Age         0
Fare        0
dtype: int64

In [69]:
df.shape

(714, 3)

In [70]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [71]:
X = df.drop('Survived', axis=1)
y = df['Survived']

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [73]:
X_train.head(5)

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542
253,30.0,16.1000
719,33.0,7.7750
666,25.0,13.0000


In [74]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

In [75]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.6153846153846154

In [76]:
np.mean(cross_val_score(DecisionTreeClassifier(), X, y, cv=10, scoring='accuracy'))

0.630281690140845

In [77]:
from sklearn.preprocessing import KBinsDiscretizer
kbin_age = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')

In [78]:
trf = ColumnTransformer([
    ('first', kbin_age, [0]),
    ('second', kbin_fare, [1])
])

In [79]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [80]:
trf.named_transformers_

{'first': KBinsDiscretizer(encode='ordinal', n_bins=10),
 'second': KBinsDiscretizer(encode='ordinal', n_bins=10)}

In [81]:
trf.named_transformers_['second'].n_bins_

array([10])

In [82]:
trf.named_transformers_['second'].bin_edges_

array([array([  0.    ,   7.75  ,   7.8958,   9.225 ,  13.    ,  15.75  ,
               26.    ,  29.125 ,  51.4792,  82.1708, 512.3292])         ],
      dtype=object)

In [83]:
output = pd.DataFrame({
    'age':X_train['Age'],
    'age_trf':X_train_trf[:,0],

    'fare':X_train['Fare'],
    'fare_trf':X_train_trf[:,1]
})

In [84]:
output.sample(5)

,age,age_trf,fare,fare_trf
377,27.0,4.0,211.5000,9.0
1,38.0,7.0,71.2833,8.0
821,27.0,4.0,8.6625,2.0
184,4.0,0.0,22.0250,5.0
843,34.5,6.0,6.4375,0.0


In [85]:
output['age_labels'] = pd.cut(
    x=X_train['Age'],
    bins=trf.named_transformers_['first'].bin_edges_[0].tolist()
    )

output['fare_label'] = pd.cut(
    x = X_train['Fare'],
    bins = trf.named_transformers_['second'].bin_edges_[0].tolist()
)

In [86]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_label
478,22.0,3.0,7.5208,0.0,"(19.0, 22.0]","(0.0, 7.75]"
173,21.0,2.0,7.9250,2.0,"(19.0, 22.0]","(7.896, 9.225]"
881,33.0,6.0,7.8958,2.0,"(32.0, 36.0]","(7.75, 7.896]"
642,2.0,0.0,27.9000,6.0,"(0.42, 14.0]","(26.0, 29.125]"
512,36.0,7.0,26.2875,6.0,"(32.0, 36.0]","(26.0, 29.125]"


In [87]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf, y_train)
y_pred2 = clf.predict(X_test_trf)

In [88]:
accuracy_score(y_test, y_pred2)

0.6223776223776224